In [71]:
import numpy as np
import pandas as pd
import time
from sklearn.metrics import f1_score, classification_report, confusion_matrix

In [72]:
class SoftmaxRegression:
    def __init__(self, learning_rate=0.1, epochs=1000):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.W = None
        self.b = None

    def _softmax(self, z):
        exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)

    def _one_hot(self, y, num_classes):
        return np.eye(num_classes)[y]

    def _cross_entropy_loss(self, y_true, y_pred):
        return -np.mean(np.sum(y_true * np.log(y_pred + 1e-15), axis=1))

    def fit(self, X, y):
        num_samples, num_features = X.shape
        num_classes = np.max(y) + 1

        # Initialize weights and bias
        self.W = np.random.randn(num_features, num_classes) * 0.01
        self.b = np.zeros((1, num_classes))

        # One-hot encode labels
        Y_onehot = self._one_hot(y, num_classes)

        for epoch in range(self.epochs):
            # Forward pass
            logits = np.dot(X, self.W) + self.b
            probs = self._softmax(logits)

            # Loss (for monitoring)
            loss = self._cross_entropy_loss(Y_onehot, probs)

            # Backward pass - CORRECTED GRADIENTS
            grad_logits = probs - Y_onehot
            grad_W = np.dot(X.T, grad_logits) / num_samples
            grad_b = np.sum(grad_logits, axis=0, keepdims=True) / num_samples

            # Update weights
            self.W -= self.learning_rate * grad_W
            self.b -= self.learning_rate * grad_b

            if epoch % 100 == 0:
                print(f"Epoch {epoch}: Loss = {loss:.4f}")

    def predict_proba(self, X):
        logits = np.dot(X, self.W) + self.b
        return self._softmax(logits)

    def predict(self, X):
        probs = self.predict_proba(X)
        return np.argmax(probs, axis=1)

In [73]:
# Load and preprocess data
train_data = pd.read_csv('MNIST_train.csv')
val_data = pd.read_csv('MNIST_validation.csv')
print("Loaded data successfully.")

Loaded data successfully.


In [74]:
print(train_data.shape)
print(val_data.shape)

(10002, 786)
(2499, 786)


In [75]:
train_data

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28,even
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9997,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9998,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9999,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10000,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [76]:
X_train = train_data.iloc[:, 1:785].values / 255.0
y_train = train_data.iloc[:, 0].values

X_val = val_data.iloc[:, 1:785].values / 255.0
y_val = val_data.iloc[:, 0].values

print(f"Training set: {X_train.shape}")
print(f"Validation set: {X_val.shape}")

Training set: (10002, 784)
Validation set: (2499, 784)


In [77]:
# Test Softmax Regression
print("="*60)
print("SOFTMAX REGRESSION COMPLETE TESTING")
print("="*60)

# Initialize model
model = SoftmaxRegression(learning_rate=0.1, epochs=500)

# Training
print("\n🚀 Starting training...")
start_time = time.time()

model.fit(X_train, y_train)

training_time = time.time() - start_time
print(f"\n✅ Training completed in {training_time:.2f} seconds ({training_time/60:.2f} minutes)")

# Validation
print("\n🎯 Making predictions on validation set...")
pred_start = time.time()

y_pred = model.predict(X_val)

pred_time = time.time() - pred_start
print(f"✅ Prediction completed in {pred_time:.2f} seconds")

SOFTMAX REGRESSION COMPLETE TESTING

🚀 Starting training...
Epoch 0: Loss = 2.3014
Epoch 100: Loss = 0.6132
Epoch 200: Loss = 0.4918
Epoch 300: Loss = 0.4406
Epoch 400: Loss = 0.4105

✅ Training completed in 9.27 seconds (0.15 minutes)

🎯 Making predictions on validation set...
✅ Prediction completed in 0.01 seconds


In [78]:
# Calculate metrics using sklearn
print("\n" + "="*60)
print("PERFORMANCE METRICS")
print("="*60)

# F1 Scores
f1_macro = f1_score(y_val, y_pred, average='macro')
f1_weighted = f1_score(y_val, y_pred, average='weighted')
f1_per_class = f1_score(y_val, y_pred, average=None)

accuracy = np.mean(y_pred == y_val)

print(f"F1 Score (Macro): {f1_macro:.4f}")
print(f"F1 Score (Weighted): {f1_weighted:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Total Training Time: {training_time:.2f}s ({training_time/60:.2f}min)")
print(f"Prediction Time: {pred_time:.2f}s")

# Time analysis for ensemble planning
remaining_time = 300 - training_time  # 5 minutes = 300 seconds
print(f"\n⏰ TIME ANALYSIS FOR ENSEMBLE PLANNING:")
print(f"Softmax used: {training_time:.2f}s")
print(f"Remaining budget: {remaining_time:.2f}s ({remaining_time/60:.2f}min)")

if remaining_time > 60:
    print("✅ Excellent! You have plenty of time for 2-3 additional models.")
elif remaining_time > 0:
    print("⚠️  Tight! You can add 1-2 fast models only.")
else:
    print("❌ Time exceeded! Need to reduce epochs or use data subset.")


PERFORMANCE METRICS
F1 Score (Macro): 0.8892
F1 Score (Weighted): 0.8902
Accuracy: 0.8908
Total Training Time: 9.27s (0.15min)
Prediction Time: 0.01s

⏰ TIME ANALYSIS FOR ENSEMBLE PLANNING:
Softmax used: 9.27s
Remaining budget: 290.73s (4.85min)
✅ Excellent! You have plenty of time for 2-3 additional models.


In [79]:
# Sample predictions visualization
print("\n👀 Sample Predictions (first 20 validation samples):")
sample_indices = range(20)
sample_actual = y_val[sample_indices]
sample_pred = y_pred[sample_indices]

for i, (actual, pred) in enumerate(zip(sample_actual, sample_pred)):
    status = "✅" if actual == pred else "❌"
    print(f"Sample {i:2d}: Actual={actual}, Predicted={pred} {status}")

# Error analysis
errors = y_pred != y_val
error_rate = np.mean(errors)
print(f"\n📈 Error Rate: {error_rate:.4f} ({np.sum(errors)} errors out of {len(y_val)})")


👀 Sample Predictions (first 20 validation samples):
Sample  0: Actual=4, Predicted=4 ✅
Sample  1: Actual=6, Predicted=6 ✅
Sample  2: Actual=7, Predicted=7 ✅
Sample  3: Actual=2, Predicted=2 ✅
Sample  4: Actual=5, Predicted=5 ✅
Sample  5: Actual=6, Predicted=6 ✅
Sample  6: Actual=4, Predicted=4 ✅
Sample  7: Actual=8, Predicted=8 ✅
Sample  8: Actual=4, Predicted=4 ✅
Sample  9: Actual=5, Predicted=5 ✅
Sample 10: Actual=1, Predicted=1 ✅
Sample 11: Actual=1, Predicted=1 ✅
Sample 12: Actual=3, Predicted=3 ✅
Sample 13: Actual=3, Predicted=2 ❌
Sample 14: Actual=8, Predicted=8 ✅
Sample 15: Actual=6, Predicted=6 ✅
Sample 16: Actual=9, Predicted=9 ✅
Sample 17: Actual=4, Predicted=4 ✅
Sample 18: Actual=6, Predicted=6 ✅
Sample 19: Actual=5, Predicted=5 ✅

📈 Error Rate: 0.1092 (273 errors out of 2499)


In [80]:
import numpy as np
import time
from collections import Counter
from sklearn.metrics import f1_score, classification_report

class KNN:
    def __init__(self, k=5):
        self.k = k
        self.X_train = None
        self.y_train = None
        
    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
        
    def predict(self, X):
        predictions = []
        for i, x in enumerate(X):
            # Compute distances to all training points
            distances = np.sqrt(np.sum((self.X_train - x) ** 2, axis=1))
            
            # Get k nearest neighbors
            nearest_indices = np.argsort(distances)[:self.k]
            nearest_labels = self.y_train[nearest_indices]
            
            # Majority vote
            most_common = Counter(nearest_labels).most_common(1)[0][0]
            predictions.append(most_common)
            
            if i % 500 == 0:
                print(f"Processed {i}/{len(X)} samples...")
                
        return np.array(predictions)

In [81]:
# Test KNN on FULL dataset
print("="*60)
print("KNN MODEL TESTING - FULL DATASET")
print("="*60)

# Initialize KNN
knn = KNN(k=5)

# Training (KNN just stores the data)
print("\n🚀 Starting KNN 'training' (storing data)...")
train_start = time.time()
knn.fit(X_train, y_train)  # Use FULL training data
training_time = time.time() - train_start
print(f"✅ 'Training' completed in {training_time:.4f} seconds")

# Prediction (this is where KNN takes time)
print("\n🎯 Making predictions on validation set...")
pred_start = time.time()
y_pred_knn = knn.predict(X_val)  # Predict on FULL validation set
pred_time = time.time() - pred_start

print(f"✅ Prediction completed in {pred_time:.2f} seconds")

KNN MODEL TESTING - FULL DATASET

🚀 Starting KNN 'training' (storing data)...
✅ 'Training' completed in 0.0000 seconds

🎯 Making predictions on validation set...
Processed 0/2499 samples...
Processed 500/2499 samples...
Processed 1000/2499 samples...
Processed 1500/2499 samples...
Processed 2000/2499 samples...
✅ Prediction completed in 176.12 seconds


In [82]:
import numpy as np
import time
from collections import Counter
from sklearn.metrics import f1_score, classification_report

class OptimizedWeightedKNN:
    def __init__(self, k=5):
        self.k = k
        self.X_train = None
        self.y_train = None
        
    def fit(self, X, y):
        # Convert to float32 for faster computation
        self.X_train = X.astype(np.float32)
        self.y_train = y.astype(np.int32)
        
    def predict(self, X):
        # Convert to float32 for faster computation
        X = X.astype(np.float32)
        predictions = np.empty(X.shape[0], dtype=np.int32)
        
        # Precompute squared norms for training data
        train_norms = np.sum(self.X_train ** 2, axis=1)
        
        for i in range(X.shape[0]):
            # Use squared Euclidean distance
            test_norm = np.sum(X[i] ** 2)
            dot_product = np.dot(self.X_train, X[i])
            squared_distances = train_norms + test_norm - 2 * dot_product
            
            # Get k nearest neighbors
            nearest_indices = np.argpartition(squared_distances, self.k)[:self.k]
            nearest_distances = squared_distances[nearest_indices]
            nearest_labels = self.y_train[nearest_indices]
            
            # Convert squared distances to actual distances for weighting
            distances = np.sqrt(nearest_distances + 1e-8)  # Add small epsilon to avoid division by zero
            
            # Calculate weights (inverse distance weighting)
            weights = 1.0 / (distances + 1e-8)
            
            # Weighted majority voting
            weighted_votes = {}
            for label, weight in zip(nearest_labels, weights):
                if label in weighted_votes:
                    weighted_votes[label] += weight
                else:
                    weighted_votes[label] = weight
            
            # Find label with highest total weight
            predictions[i] = max(weighted_votes.items(), key=lambda x: x[1])[0]
            
        return predictions

class BatchOptimizedWeightedKNN:
    def __init__(self, k=5, batch_size=100):
        self.k = k
        self.batch_size = batch_size
        self.X_train = None
        self.y_train = None
        
    def fit(self, X, y):
        self.X_train = X.astype(np.float32)
        self.y_train = y.astype(np.int32)
        
    def predict(self, X):
        X = X.astype(np.float32)
        predictions = np.empty(X.shape[0], dtype=np.int32)
        
        # Precompute squared norms for training data
        train_norms = np.sum(self.X_train ** 2, axis=1)
        
        # Process in batches
        for batch_start in range(0, X.shape[0], self.batch_size):
            batch_end = min(batch_start + self.batch_size, X.shape[0])
            X_batch = X[batch_start:batch_end]
            
            # Vectorized distance computation
            test_norms = np.sum(X_batch ** 2, axis=1)
            dot_products = np.dot(self.X_train, X_batch.T)
            squared_distances = train_norms[:, np.newaxis] + test_norms - 2 * dot_products
            
            # Get k nearest neighbors for each batch sample
            nearest_indices = np.argpartition(squared_distances, self.k, axis=0)[:self.k]
            nearest_squared_distances = np.take_along_axis(squared_distances, nearest_indices, axis=0)
            nearest_labels = self.y_train[nearest_indices]
            
            # Convert to actual distances and compute weights
            distances = np.sqrt(nearest_squared_distances + 1e-8)
            weights = 1.0 / (distances + 1e-8)
            
            # Weighted majority voting for each sample in batch
            for j in range(batch_end - batch_start):
                label_weights = {}
                for idx in range(self.k):
                    label = nearest_labels[idx, j]
                    weight = weights[idx, j]
                    label_weights[label] = label_weights.get(label, 0.0) + weight
                
                predictions[batch_start + j] = max(label_weights.items(), key=lambda x: x[1])[0]
                
        return predictions

# Test the optimized weighted KNN
print("Testing OptimizedWeightedKNN...")
weighted_knn = BatchOptimizedWeightedKNN(k=5, batch_size=200)

start_time = time.time()
weighted_knn.fit(X_train, y_train)
training_time = time.time() - start_time

pred_start = time.time()
y_pred_weighted = weighted_knn.predict(X_val)
pred_time = time.time() - pred_start

f1_weighted = f1_score(y_val, y_pred_weighted, average='weighted')

print(f"WeightedKNN Results:")
print(f"F1 Score: {f1_weighted:.4f} ")
print(f"Training Time: {training_time:.4f}s")
print(f"Prediction Time: {pred_time:.4f}s")
print(f"Total Time: {training_time + pred_time:.4f}s")

Testing OptimizedWeightedKNN...
WeightedKNN Results:
F1 Score: 0.9515 
Training Time: 0.0187s
Prediction Time: 1.2661s
Total Time: 1.2847s


In [83]:
import numpy as np
import time
from collections import Counter
from sklearn.metrics import f1_score, classification_report

class KNN:
    def __init__(self, k=5):
        self.k = k
        self.X_train = None
        self.y_train = None
        
    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
        
    def predict(self, X):
        predictions = []
        for i, x in enumerate(X):
            # Compute distances to all training points
            distances = np.sqrt(np.sum((self.X_train - x) ** 2, axis=1))
            
            # Get k nearest neighbors
            nearest_indices = np.argsort(distances)[:self.k]
            nearest_labels = self.y_train[nearest_indices]
            
            # Majority vote
            most_common = Counter(nearest_labels).most_common(1)[0][0]
            predictions.append(most_common)
            
            if i % 500 == 0:
                print(f"Processed {i}/{len(X)} samples...")
                
        return np.array(predictions)

In [84]:
# Calculate metrics
print("\n" + "="*60)
print("KNN PERFORMANCE METRICS")
print("="*60)

f1_weighted_knn = f1_score(y_val, y_pred_knn, average='weighted')



print(f"F1 Score (Weighted): {f1_weighted_knn:.4f}")
print(f"Training Time: {training_time:.4f}s")
print(f"Prediction Time: {pred_time:.2f}s")

# Compare with Softmax
print(f"\n🆚 COMPARISON WITH SOFTMAX:")
print(f"Softmax F1 (Weighted): 0.8902")
print(f"KNN F1 (Weighted):     {f1_weighted_knn:.4f}")
print(f"Softmax Training: 7.92s")
print(f"KNN Training: {training_time:.4f}s")
print(f"KNN Prediction: {pred_time:.2f}s")

# Time analysis
total_knn_time = training_time + pred_time
remaining_time = 300 - 7.92 - total_knn_time  # Subtract Softmax + KNN times
print(f"\n⏰ REMAINING TIME BUDGET: {remaining_time:.2f}s ({remaining_time/60:.2f}min)")


KNN PERFORMANCE METRICS
F1 Score (Weighted): 0.9523
Training Time: 0.0187s
Prediction Time: 1.27s

🆚 COMPARISON WITH SOFTMAX:
Softmax F1 (Weighted): 0.8902
KNN F1 (Weighted):     0.9523
Softmax Training: 7.92s
KNN Training: 0.0187s
KNN Prediction: 1.27s

⏰ REMAINING TIME BUDGET: 290.80s (4.85min)


In [85]:
# Quick check - do KNN and Softmax make different errors?
common_errors = np.sum((y_pred != y_val) & (y_pred_knn != y_val))
different_errors = np.sum((y_pred != y_val) ^ (y_pred_knn != y_val))
print(f"Common errors: {common_errors}")
print(f"Different errors: {different_errors}")

Common errors: 85
Different errors: 222


In [86]:
# ----------- Decision Tree Node -----------
class DecisionTreeNode:
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, value=None):
        self.feature_index = feature_index  # index of feature to split on
        self.threshold = threshold          # threshold value to split
        self.left = left                    # left subtree
        self.right = right                  # right subtree
        self.value = value                  # class label for leaf nodes

    def is_leaf_node(self):
        # returns true if this node hold a value
        return self.value is not None
# ----------- Decision Tree Classifier -----------s
class DecisionTreeClassifier:
    def __init__(self, max_depth=10, min_samples_split=2, feature_indices = None):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.root = None
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        # This is an extra line from earlier
        # the indices of features to be used are passed as argument
        self.feature_indices = feature_indices
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

    def fit(self, X, y):
        #print ('in fit function, type of X is', type(X))
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        # This is an different from earlier
        # if subset of features is not defined
        if not self.feature_indices:
            self.feature_indices = list(np.arange(len(X[0])))
            assert len(self.feature_indices) > 1, 'less than 2 features, tree building may fail'
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        self.root = self._build_tree(X, y)

    # additional argument 
    def _build_tree(self, X, y, depth=0):
        num_classes = len(set(y))
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        # base case 1
        # This is an different from earlier
        num_samples = len(y)
        if (num_samples == 0):
            return None
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        # base case 2
        # stopping conditions
        if (depth >= self.max_depth or
            num_classes == 1 or
            num_samples < self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return DecisionTreeNode(value=leaf_value)
            
        # greedy search for best split
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        # feat_idxs instead of num_features in this line
        best_feat, best_thresh = self._best_split(X, y)
        #print ('best feat and best thresh are', best_feat, best_thresh)
        # base case 3 - best split not found 
        if best_feat is None:
            leaf_value = self._most_common_label(y)
            #print ('best feat is none, returning common label', leaf_value)
            return DecisionTreeNode(value=leaf_value)

        # recursive case 
        # split is found
        left_idx = X[:, best_feat] <= best_thresh
        right_idx = X[:, best_feat] > best_thresh
        #print ('left_idx and right idx are',sum(left_idx), sum(right_idx))

        # if one of the parts is empty
        if np.sum(left_idx) == 0 or np.sum(right_idx) == 0:
            # If either side is empty, return a leaf node with the most common label
            leaf_value = self.most_common_label(y)
            return DecisionTreeNode(value=leaf_value)
            
        left = self._build_tree(X[left_idx], y[left_idx], depth + 1)
        right = self._build_tree(X[right_idx], y[right_idx], depth + 1)
        return DecisionTreeNode(feature_index=best_feat, threshold=best_thresh, left=left, right=right)

    def _best_split(self, X, y):
        best_gain = 0
        split_idx, split_thresh = None, None

        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        # feat_idxs instead of num_features in this line
        for feat_idx in self.feature_indices:
            #print ('feat idx is', feat_idx)
            #print ('type of X is ', type(X))
            thresholds = np.unique(X[:, feat_idx])
            #print ('thresholds', thresholds)
            for thresh in thresholds:
                #print ('each thresh is', thresh)
                gain = self._gini_gain(y, X[:, feat_idx], thresh)
                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_thresh = thresh
            #print ('completed for loop')
        #print ('best split is', split_idx, split_thresh, best_gain)
        return split_idx, split_thresh

    def _gini_gain(self, y, feature_column, threshold):
        # parent gini
        parent_gini = self._gini(y)

        # generate splits
        left_idx = feature_column <= threshold
        right_idx = feature_column > threshold

        if len(y[left_idx]) == 0 or len(y[right_idx]) == 0:
            return 0

        # weighted avg. gini of children
        n = len(y)
        n_left, n_right = len(y[left_idx]), len(y[right_idx])
        gini_left = self._gini(y[left_idx])
        gini_right = self._gini(y[right_idx])
        child_gini = (n_left / n) * gini_left + (n_right / n) * gini_right

        # gini gain
        return parent_gini - child_gini

    def _gini(self, y):
        counts = np.bincount(y)
        probabilities = counts / len(y)
        return 1.0 - sum(p**2 for p in probabilities if p > 0)

    def _most_common_label(self, y):
        if set(y) is None:
            return None
        #print ('unique y values are', set(y))
        counter = Counter(y)
        #print ('in most common label', counter)
        most_common = counter.most_common(1)[0][0]
        #print ('in most common label', counter, most_common)
        return most_common

    def predict(self, X):
        return np.array([self._predict(inputs, self.root) for inputs in X])

    def _predict(self, inputs, node):
        # base case
        if node.is_leaf_node():
            return node.value
        # recursive calling of left and right branches
        if inputs[node.feature_index] <= node.threshold:
            return self._predict(inputs, node.left)
        else:
            return self._predict(inputs, node.right)
# ----------- Random Forest Classifier -----------
class RandomForest:
    def __init__(self, n_trees=10, max_depth=10, min_samples_split=2, max_features=None):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_features = max_features
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        self.max_features = self.max_features or len(X[0])
        
        for tci in range(self.n_trees):
            print ('~~~~~~~~~~~~~~~~~~~~~~~')
            print ('builiding tree', tci)
            
            X_sample, y_sample = self._bootstrap_sample(X, y)
            
            # selection feature indices
            feature_indices = random.sample(range(len(X[0])), self.max_features)
            print ('in tree, feature indices are', tci, feature_indices)
            
            tree = DecisionTreeClassifier(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                feature_indices = feature_indices
            )
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)
            print ('completed building tree', tci)
            print ('~~~~~~~~~~~~~~~~~~~~~~~')

    def _bootstrap_sample(self, X, y):
        #print ('in bootstrap, type of X is', type(X))
        n_samples = len(X)
        indices = [random.randint(0, n_samples - 1) for _ in range(n_samples)]
        #print ('length of samples and indices is', n_samples, len(indices))
        return np.array([X[i] for i in indices]), np.array([y[i] for i in indices])

    def predict(self, X):
        tree_preds = [tree.predict(X) for tree in self.trees]
        print ('before transformation, tree_preds, length and type', len(tree_preds), type(tree_preds), np.shape(tree_preds))
        tree_preds = list(zip(*tree_preds))
        print ('after transformation, tree_preds, length and type', len(tree_preds), type(tree_preds), np.shape(tree_preds))
        return [self._most_common_label(preds) for preds in tree_preds]

    def _most_common_label(self, labels):
        return Counter(labels).most_common(1)[0][0]

In [87]:
import random
from collections import Counter

# Test Random Forest on FULL dataset
print("="*60)
print("RANDOM FOREST MODEL TESTING - FULL DATASET")
print("="*60)

# Initialize Random Forest with smaller settings for speed
rf = RandomForest(
    n_trees=5,           # Smaller number of trees for speed
    max_depth=10,        # Limit depth
    min_samples_split=20, # Larger min samples for speed
    max_features=100     # Use only 100 features per tree for speed
)

# Training
print("\n🚀 Starting Random Forest training...")
train_start = time.time()
rf.fit(X_train, y_train)
training_time = time.time() - train_start
print(f"✅ Training completed in {training_time:.2f} seconds")

# Prediction
print("\n🎯 Making predictions on validation set...")
pred_start = time.time()
y_pred_rf = rf.predict(X_val)
pred_time = time.time() - pred_start

print(f"✅ Prediction completed in {pred_time:.2f} seconds")

RANDOM FOREST MODEL TESTING - FULL DATASET

🚀 Starting Random Forest training...
~~~~~~~~~~~~~~~~~~~~~~~
builiding tree 0
in tree, feature indices are 0 [754, 301, 611, 235, 328, 647, 381, 547, 443, 659, 291, 697, 32, 615, 486, 251, 165, 102, 579, 682, 341, 254, 442, 481, 266, 427, 391, 610, 404, 221, 274, 304, 332, 42, 738, 529, 259, 664, 249, 613, 530, 409, 347, 238, 513, 280, 326, 179, 540, 531, 22, 535, 35, 227, 142, 211, 466, 93, 623, 205, 480, 686, 639, 177, 464, 641, 576, 489, 702, 581, 164, 345, 424, 101, 209, 353, 773, 721, 484, 572, 690, 299, 420, 377, 116, 521, 210, 445, 365, 394, 358, 661, 124, 414, 150, 747, 618, 778, 582, 441]
completed building tree 0
~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~
builiding tree 1
in tree, feature indices are 1 [239, 523, 547, 658, 260, 778, 278, 454, 327, 606, 369, 465, 17, 690, 370, 618, 684, 301, 521, 226, 609, 227, 575, 435, 631, 276, 308, 137, 641, 174, 122, 358, 8, 434, 645, 249, 0, 6, 636, 402, 204, 79, 568, 508, 607, 351, 682, 7

In [88]:
# Calculate metrics
print("\n" + "="*60)
print("RANDOM FOREST PERFORMANCE METRICS")
print("="*60)

f1_weighted_rf = f1_score(y_val, y_pred_rf, average='weighted')



print(f"F1 Score (Weighted): {f1_weighted_rf:.4f}")
print(f"Training Time: {training_time:.2f}s")
print(f"Prediction Time: {pred_time:.2f}s")

# Compare with previous models
print(f"\n🆚 COMPARISON WITH ALL MODELS:")
print(f"Softmax F1: 0.8899")
print(f"KNN F1:     0.9523") 
print(f"Random Forest F1: {f1_weighted_rf:.4f}")

# Time analysis
total_rf_time = training_time + pred_time
total_used = 8.11 + 163.38 + total_rf_time
remaining_time = 300 - total_used
print(f"\n⏰ REMAINING TIME BUDGET: {remaining_time:.2f}s ({remaining_time/60:.2f}min)")


RANDOM FOREST PERFORMANCE METRICS
F1 Score (Weighted): 0.8623
Training Time: 156.67s
Prediction Time: 0.04s

🆚 COMPARISON WITH ALL MODELS:
Softmax F1: 0.8899
KNN F1:     0.9523
Random Forest F1: 0.8623

⏰ REMAINING TIME BUDGET: -28.19s (-0.47min)


In [89]:
class GaussianNB:
    def __init__(self):
        self.means = None
        self.variances = None
        self.priors = None
        self.classes = None
        
    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.classes = np.unique(y)
        n_classes = len(self.classes)
        
        self.means = np.zeros((n_classes, n_features))
        self.variances = np.zeros((n_classes, n_features))
        self.priors = np.zeros(n_classes)
        
        for i, c in enumerate(self.classes):
            X_c = X[y == c]
            self.means[i] = X_c.mean(axis=0)
            self.variances[i] = X_c.var(axis=0) + 1e-9  # Smoothing to avoid division by zero
            self.priors[i] = X_c.shape[0] / n_samples
            
    def predict(self, X):
        posteriors = []
        for i in range(len(self.classes)):
            prior = np.log(self.priors[i])
            # Gaussian Naive Bayes: log(P(x|y)) = sum(log( GaussianPDF(x_i|mean_i, var_i) ))
            # Simplified: -0.5 * sum( (x - mean)^2 / variance + log(2*pi*variance) )
            nll = -0.5 * np.sum(np.log(2. * np.pi * self.variances[i]))
            nll -= 0.5 * np.sum(((X - self.means[i]) ** 2) / self.variances[i], axis=1)
            posterior = prior + nll
            posteriors.append(posterior)
        
        posteriors = np.array(posteriors).T
        return self.classes[np.argmax(posteriors, axis=1)]

# Test Naive Bayes
print("="*60)
print("NAIVE BAYES MODEL TESTING")
print("="*60)

# Initialize Naive Bayes
nb = GaussianNB()

# Training
print("\n🚀 Starting Naive Bayes training...")
train_start = time.time()
nb.fit(X_train, y_train)
training_time = time.time() - train_start
print(f"✅ Training completed in {training_time:.4f} seconds")

# Prediction
print("\n🎯 Making predictions on validation set...")
pred_start = time.time()
y_pred_nb = nb.predict(X_val)
pred_time = time.time() - pred_start

print(f"✅ Prediction completed in {pred_time:.4f} seconds")

NAIVE BAYES MODEL TESTING

🚀 Starting Naive Bayes training...
✅ Training completed in 0.1335 seconds

🎯 Making predictions on validation set...
✅ Prediction completed in 0.2388 seconds


In [90]:
# Calculate metrics
print("\n" + "="*60)
print("NAIVE BAYES PERFORMANCE METRICS")
print("="*60)

f1_macro_nb = f1_score(y_val, y_pred_nb, average='macro')
f1_weighted_nb = f1_score(y_val, y_pred_nb, average='weighted')
accuracy_nb = np.mean(y_pred_nb == y_val)

print(f"F1 Score (Macro): {f1_macro_nb:.4f}")
print(f"F1 Score (Weighted): {f1_weighted_nb:.4f}")
print(f"Accuracy: {accuracy_nb:.4f}")
print(f"Training Time: {training_time:.4f}s")
print(f"Prediction Time: {pred_time:.4f}s")

# Compare with all models
print(f"\n🆚 COMPARISON WITH ALL MODELS:")
print(f"Softmax F1: 0.8899")
print(f"KNN F1:     0.9523")
print(f"Random Forest F1: 0.8503")
print(f"Naive Bayes F1:   {f1_weighted_nb:.4f}")

# Time analysis
total_nb_time = training_time + pred_time
total_used = 8.11 + 163.38 + 134.94 + total_nb_time
remaining_time = 300 - total_used
print(f"\n⏰ REMAINING TIME BUDGET: {remaining_time:.2f}s ({remaining_time/60:.2f}min)")


NAIVE BAYES PERFORMANCE METRICS
F1 Score (Macro): 0.5351
F1 Score (Weighted): 0.5424
Accuracy: 0.5714
Training Time: 0.1335s
Prediction Time: 0.2388s

🆚 COMPARISON WITH ALL MODELS:
Softmax F1: 0.8899
KNN F1:     0.9523
Random Forest F1: 0.8503
Naive Bayes F1:   0.5424

⏰ REMAINING TIME BUDGET: -6.80s (-0.11min)


In [ ]:
class MultiClassSVM:
    def __init__(self, learning_rate=0.001, lambda_param=0.01, n_iters=1000, n_classes=10):
        self.lr = learning_rate
        self.lambda_param = lambda_param
        self.n_iters = n_iters
        self.n_classes = n_classes
        self.W = None  # Weight matrix for all classes
        self.b = None  # Bias vector for all classes

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.W = np.zeros((self.n_classes, n_features))
        self.b = np.zeros(self.n_classes)

        # One-vs-Rest approach
        for class_idx in range(self.n_classes):
            print(f"Training SVM for class {class_idx}...")
            # Convert to binary labels: current class vs all others
            y_binary = np.where(y == class_idx, 1, -1)
            
            w = np.zeros(n_features)
            b = 0

            for _ in range(self.n_iters):
                for i, x_i in enumerate(X):
                    condition = y_binary[i] * (np.dot(x_i, w) - b) >= 1
                    if condition:
                        w -= self.lr * (self.lambda_param * w)
                    else:
                        w -= self.lr * (self.lambda_param * w - np.dot(x_i, y_binary[i]))
                        b -= self.lr * y_binary[i]

            self.W[class_idx] = w
            self.b[class_idx] = b

    def predict(self, X):
        # For each sample, find the class with highest score
        scores = np.dot(X, self.W.T) - self.b
        return np.argmax(scores, axis=1)

# Test SVM
print("="*60)
print("SVM MODEL TESTING - ONE vs REST")
print("="*60)

# Initialize SVM with fewer iterations for speed
svm = MultiClassSVM(
    learning_rate=0.001,
    lambda_param=0.01, 
    n_iters=100,  # Reduced for speed
    n_classes=10
)

# Training
print("\n🚀 Starting SVM training (this may take a while)...")
train_start = time.time()
svm.fit(X_train, y_train)
training_time = time.time() - train_start
print(f"✅ Training completed in {training_time:.2f} seconds")

# Prediction
print("\n🎯 Making predictions on validation set...")
pred_start = time.time()
y_pred_svm = svm.predict(X_val)
pred_time = time.time() - pred_start

print(f"✅ Prediction completed in {pred_time:.4f} seconds")

SVM MODEL TESTING - ONE vs REST

🚀 Starting SVM training (this may take a while)...
Training SVM for class 0...
Training SVM for class 1...
Training SVM for class 2...
Training SVM for class 3...
Training SVM for class 4...
Training SVM for class 5...
Training SVM for class 6...
Training SVM for class 7...
Training SVM for class 8...
Training SVM for class 9...
✅ Training completed in 202.50 seconds

🎯 Making predictions on validation set...
✅ Prediction completed in 0.0068 seconds


In [ ]:
# Calculate metrics
print("\n" + "="*60)
print("SVM PERFORMANCE METRICS")
print("="*60)

f1_macro_svm = f1_score(y_val, y_pred_svm, average='macro')
f1_weighted_svm = f1_score(y_val, y_pred_svm, average='weighted')
accuracy_svm = np.mean(y_pred_svm == y_val)

print(f"F1 Score (Macro): {f1_macro_svm:.4f}")
print(f"F1 Score (Weighted): {f1_weighted_svm:.4f}")
print(f"Accuracy: {accuracy_svm:.4f}")
print(f"Training Time: {training_time:.2f}s")
print(f"Prediction Time: {pred_time:.4f}s")

# Compare with all models
print(f"\n🆚 COMPARISON WITH ALL MODELS:")
print(f"KNN F1:          0.9523")
print(f"Softmax F1:      0.8899")
print(f"Random Forest F1: 0.8503")
print(f"Naive Bayes F1:   0.5424")
print(f"SVM F1:          {f1_weighted_svm:.4f}")

# Time analysis
total_svm_time = training_time + pred_time
total_used = 8.11 + 163.38 + 134.94 + total_svm_time
remaining_time = 300 - total_used
print(f"\n⏰ REMAINING TIME BUDGET: {remaining_time:.2f}s ({remaining_time/60:.2f}min)")


SVM PERFORMANCE METRICS
F1 Score (Macro): 0.8922
F1 Score (Weighted): 0.8934
Accuracy: 0.8948
Training Time: 202.50s
Prediction Time: 0.0068s

🆚 COMPARISON WITH ALL MODELS:
KNN F1:          0.9523
Softmax F1:      0.8899
Random Forest F1: 0.8503
Naive Bayes F1:   0.5424
SVM F1:          0.8934

⏰ REMAINING TIME BUDGET: -208.94s (-3.48min)


In [101]:
import numpy as np
import time
from sklearn.metrics import f1_score

class XGBoostTree:
    def __init__(self, max_depth=3, reg_lambda=1.0, gamma=0.0, max_bins=50):
        self.max_depth = max_depth
        self.reg_lambda = reg_lambda
        self.gamma = gamma
        self.max_bins = max_bins
        self.feature = None
        self.threshold = None
        self.left = None
        self.right = None
        self.leaf_value = None

    def _calc_leaf_value(self, G, H):
        return -G / (H + self.reg_lambda)

    def _calc_gain(self, G_left, H_left, G_right, H_right, G_total, H_total):
        def _term(G, H):
            return G * G / (H + self.reg_lambda)
        return 0.5 * (_term(G_left, H_left) + _term(G_right, H_right) - _term(G_total, H_total)) - self.gamma

    def _find_best_split(self, X, g, h, sample_indices, feature_thresholds):
        G_total = np.sum(g[sample_indices])
        H_total = np.sum(h[sample_indices])
        
        best_gain = 0.0
        best_feature = None
        best_threshold = None

        for feat, thresholds in enumerate(feature_thresholds):
            if thresholds is None:
                continue

            x_col = X[sample_indices, feat]
            g_vals = g[sample_indices]
            h_vals = h[sample_indices]

            # Sort samples by feature value for efficient cumulative sums
            sorted_idx = np.argsort(x_col)
            x_sorted = x_col[sorted_idx]
            g_sorted = g_vals[sorted_idx]
            h_sorted = h_vals[sorted_idx]

            # Precompute cumulative sums
            cumsum_g = np.cumsum(g_sorted)
            cumsum_h = np.cumsum(h_sorted)

            # Try each threshold
            for th in thresholds:
                # Find split point: first index where x > th
                split_pos = np.searchsorted(x_sorted, th, side='right')
                if split_pos == 0 or split_pos == len(x_sorted):
                    continue

                G_left = cumsum_g[split_pos - 1]
                H_left = cumsum_h[split_pos - 1]
                G_right = G_total - G_left
                H_right = H_total - H_left

                gain = self._calc_gain(G_left, H_left, G_right, H_right, G_total, H_total)
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feat
                    best_threshold = th

        return best_feature, best_threshold, best_gain

    def _fit_recursive(self, X, g, h, sample_indices, depth, feature_thresholds):
        n_samples = len(sample_indices)
        G = np.sum(g[sample_indices])
        H = np.sum(h[sample_indices])
        self.leaf_value = self._calc_leaf_value(G, H)

        if depth >= self.max_depth or n_samples < 2:
            return

        feature, threshold, gain = self._find_best_split(X, g, h, sample_indices, feature_thresholds)

        if feature is None or gain <= 0:
            return

        self.feature = feature
        self.threshold = threshold

        left_mask = X[sample_indices, feature] <= threshold
        left_indices = sample_indices[left_mask]
        right_indices = sample_indices[~left_mask]

        if len(left_indices) == 0 or len(right_indices) == 0:
            return

        self.left = XGBoostTree(
            max_depth=self.max_depth,
            reg_lambda=self.reg_lambda,
            gamma=self.gamma,
            max_bins=self.max_bins
        )
        self.left._fit_recursive(X, g, h, left_indices, depth + 1, feature_thresholds)

        self.right = XGBoostTree(
            max_depth=self.max_depth,
            reg_lambda=self.reg_lambda,
            gamma=self.gamma,
            max_bins=self.max_bins
        )
        self.right._fit_recursive(X, g, h, right_indices, depth + 1, feature_thresholds)

    def fit(self, X, g, h, feature_thresholds):
        sample_indices = np.arange(X.shape[0])
        self._fit_recursive(X, g, h, sample_indices, 0, feature_thresholds)
        return self

    def predict(self, X):
        if self.feature is None:
            return np.full(X.shape[0], self.leaf_value)
        left_mask = X[:, self.feature] <= self.threshold
        right_mask = ~left_mask
        preds = np.empty(X.shape[0])
        preds[left_mask] = self.left.predict(X[left_mask]) if self.left else self.leaf_value
        preds[right_mask] = self.right.predict(X[right_mask]) if self.right else self.leaf_value
        return preds


class XGBoostClassifier:
    def __init__(self, n_estimators=10, learning_rate=0.1, max_depth=3, reg_lambda=1.0, gamma=0.0, max_bins=50, subsample=0.8):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.reg_lambda = reg_lambda
        self.gamma = gamma
        self.max_bins = max_bins
        self.subsample = subsample
        self.trees = []
        self.base_score = 0.0

    def _sigmoid(self, x):
        x = np.clip(x, -250, 250)
        return 1.0 / (1.0 + np.exp(-x))

    def _precompute_feature_thresholds(self, X):
        thresholds = []
        for feat in range(X.shape[1]):
            col = X[:, feat]
            if np.all(col == col[0]):
                thresholds.append(None)
                continue
            unique_vals = np.unique(col)
            if len(unique_vals) > self.max_bins:
                percentiles = np.linspace(0, 100, self.max_bins + 1)
                ths = np.percentile(col, percentiles[1:-1])
                ths = np.unique(ths)
            else:
                ths = (unique_vals[:-1] + unique_vals[1:]) / 2.0
            thresholds.append(ths if len(ths) > 0 else None)
        return thresholds

    def fit(self, X, y):
        X = np.asarray(X, dtype=np.float64)
        y = np.asarray(y, dtype=np.float64)
        y_binary = y

        # Precompute global thresholds once per dataset
        feature_thresholds = self._precompute_feature_thresholds(X)

        n_samples = X.shape[0]
        y_pred = np.full(n_samples, self.base_score, dtype=np.float64)

        for i in range(self.n_estimators):
            print(f"Building tree {i+1}/{self.n_estimators}...")
            
            # SUBSAMPLING: Randomly select subset of data for this tree
            if self.subsample < 1.0:
                subsample_size = int(self.subsample * n_samples)
                indices = np.random.choice(n_samples, subsample_size, replace=False)
                X_sub = X[indices]
                y_sub = y_binary[indices]
                y_pred_sub = y_pred[indices]
            else:
                indices = np.arange(n_samples)
                X_sub = X
                y_sub = y_binary
                y_pred_sub = y_pred

            # Calculate gradients and hessians on subsampled data
            p = self._sigmoid(y_pred_sub)
            g = p - y_sub
            h = p * (1.0 - p) + 1e-10

            # Train tree on subsampled data
            tree = XGBoostTree(
                max_depth=self.max_depth,
                reg_lambda=self.reg_lambda,
                gamma=self.gamma,
                max_bins=self.max_bins
            )
            tree.fit(X_sub, g, h, feature_thresholds)

            # Update predictions on FULL dataset
            update = tree.predict(X)
            y_pred += self.learning_rate * update
            self.trees.append(tree)

        return self

    def predict(self, X):
        X = np.asarray(X, dtype=np.float64)
        y_pred = np.full(X.shape[0], self.base_score, dtype=np.float64)
        for tree in self.trees:
            y_pred += self.learning_rate * tree.predict(X)
        proba = self._sigmoid(y_pred)
        return (proba >= 0.5).astype(int)


class MultiClassXGBoost:
    def __init__(self, n_estimators=10, learning_rate=0.1, max_depth=3, reg_lambda=1.0, gamma=0.0, max_bins=50, n_classes=10, subsample=0.8):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.reg_lambda = reg_lambda
        self.gamma = gamma
        self.max_bins = max_bins
        self.n_classes = n_classes
        self.subsample = subsample
        self.classifiers = []

    def fit(self, X, y):
        self.classifiers = []
        
        # One-vs-Rest approach for multi-class
        for class_idx in range(self.n_classes):
            print(f"Training XGBoost for class {class_idx}...")
            
            # Convert to binary labels: current class vs all others
            y_binary = (y == class_idx).astype(int)
            
            # Create and train binary classifier with subsampling
            xgb_binary = XGBoostClassifier(
                n_estimators=self.n_estimators,
                learning_rate=self.learning_rate,
                max_depth=self.max_depth,
                reg_lambda=self.reg_lambda,
                gamma=self.gamma,
                max_bins=self.max_bins,
                subsample=self.subsample
            )
            
            xgb_binary.fit(X, y_binary)
            self.classifiers.append(xgb_binary)
            
        return self

    def predict_proba(self, X):
        # Get probabilities from all classifiers
        probabilities = np.zeros((X.shape[0], self.n_classes))
        
        for class_idx, classifier in enumerate(self.classifiers):
            # Get raw scores and convert to probabilities
            raw_scores = np.zeros(X.shape[0])
            for tree in classifier.trees:
                raw_scores += classifier.learning_rate * tree.predict(X)
            
            # Sigmoid to get probabilities
            proba = 1.0 / (1.0 + np.exp(-raw_scores))
            probabilities[:, class_idx] = proba
            
        # Normalize to make sure probabilities sum to 1
        probabilities = probabilities / np.sum(probabilities, axis=1, keepdims=True)
        return probabilities

    def predict(self, X):
        probabilities = self.predict_proba(X)
        return np.argmax(probabilities, axis=1)


# Test XGBoost with subsampling for speed
print("="*60)
print("XGBOOST MODEL TESTING WITH SUBSAMPLING")
print("="*60)

# Use smaller settings with subsampling for reasonable training time
xgb = MultiClassXGBoost(
    n_estimators=20,        # Small number of trees
    learning_rate=0.1,
    max_depth=5,
    reg_lambda=1.0,
    subsample=0.1,          # Use only 60% of data for each tree - MAJOR SPEEDUP!
    n_classes=10
)

# Training
print("\n🚀 Starting XGBoost training (this may take a while)...")
train_start = time.time()
xgb.fit(X_train, y_train)
training_time = time.time() - train_start
print(f"✅ Training completed in {training_time:.2f} seconds")

# Prediction
print("\n🎯 Making predictions on validation set...")
pred_start = time.time()
y_pred_xgb = xgb.predict(X_val)
pred_time = time.time() - pred_start

print(f"✅ Prediction completed in {pred_time:.4f} seconds")

# Calculate metrics
f1_weighted_xgb = f1_score(y_val, y_pred_xgb, average='weighted')
accuracy_xgb = np.mean(y_pred_xgb == y_val)

print(f"\n📊 XGBoost Performance:")
print(f"F1 Score (Weighted): {f1_weighted_xgb:.4f}")
print(f"Accuracy: {accuracy_xgb:.4f}")
print(f"Total Time: {training_time + pred_time:.2f}s")

XGBOOST MODEL TESTING WITH SUBSAMPLING

🚀 Starting XGBoost training (this may take a while)...
Training XGBoost for class 0...
Building tree 1/20...
Building tree 2/20...
Building tree 3/20...
Building tree 4/20...
Building tree 5/20...
Building tree 6/20...
Building tree 7/20...
Building tree 8/20...
Building tree 9/20...
Building tree 10/20...
Building tree 11/20...
Building tree 12/20...
Building tree 13/20...
Building tree 14/20...
Building tree 15/20...
Building tree 16/20...
Building tree 17/20...
Building tree 18/20...
Building tree 19/20...
Building tree 20/20...
Training XGBoost for class 1...
Building tree 1/20...
Building tree 2/20...
Building tree 3/20...
Building tree 4/20...
Building tree 5/20...
Building tree 6/20...
Building tree 7/20...
Building tree 8/20...
Building tree 9/20...
Building tree 10/20...
Building tree 11/20...
Building tree 12/20...
Building tree 13/20...
Building tree 14/20...
Building tree 15/20...
Building tree 16/20...
Building tree 17/20...
Building

subsample = 0.6
n_est = 30, depth = 3 --- 10min 24sec --- 0.876
subsample = 0.1
n_est = 30, depth = 3 --- 1min 6sec --- 0.876
n_est = 50, depth = 3 --- 6min 22 sec --- 0.892
n_est = 30, depth = 4 --- 5min 10 sec --- 0.8896
n_est = 20, depth = 4 --- 

In [95]:
# Calculate metrics
print("\n" + "="*60)
print("XGBOOST PERFORMANCE METRICS")
print("="*60)

f1_macro_xgb = f1_score(y_val, y_pred_xgb, average='macro')
f1_weighted_xgb = f1_score(y_val, y_pred_xgb, average='weighted')
accuracy_xgb = np.mean(y_pred_xgb == y_val)

print(f"F1 Score (Macro): {f1_macro_xgb:.4f}")
print(f"F1 Score (Weighted): {f1_weighted_xgb:.4f}")
print(f"Accuracy: {accuracy_xgb:.4f}")
print(f"Training Time: {training_time:.2f}s")
print(f"Prediction Time: {pred_time:.4f}s")

# Compare with all models
print(f"\n🆚 COMPARISON WITH ALL MODELS:")
print(f"KNN F1:          0.9523")
print(f"SVM F1:          0.8934")
print(f"Softmax F1:      0.8899")
print(f"Random Forest F1: 0.8503")
print(f"Naive Bayes F1:   0.5424")
print(f"XGBoost F1:      {f1_weighted_xgb:.4f}")

# Time analysis
total_xgb_time = training_time + pred_time
total_used = 8.11 + 163.38 + 134.94 + 71.98 + total_xgb_time
remaining_time = 300 - total_used
print(f"\n⏰ REMAINING TIME BUDGET: {remaining_time:.2f}s ({remaining_time/60:.2f}min)")


XGBOOST PERFORMANCE METRICS
F1 Score (Macro): 0.8835
F1 Score (Weighted): 0.8843
Accuracy: 0.8844
Training Time: 644.71s
Prediction Time: 2.9117s

🆚 COMPARISON WITH ALL MODELS:
KNN F1:          0.9523
SVM F1:          0.8934
Softmax F1:      0.8899
Random Forest F1: 0.8503
Naive Bayes F1:   0.5424
XGBoost F1:      0.8843

⏰ REMAINING TIME BUDGET: -726.03s (-12.10min)


In [123]:
import numpy as np
import time
from sklearn.metrics import f1_score

class FastXGBoostTree:
    def __init__(self, max_depth=3, reg_lambda=1.0, gamma=0.0, max_bins=50):
        self.max_depth = max_depth
        self.reg_lambda = reg_lambda
        self.gamma = gamma
        self.max_bins = max_bins
        self.feature = None
        self.threshold = None
        self.left = None
        self.right = None
        self.leaf_value = None

    def _calc_leaf_value(self, G, H):
        return -G / (H + self.reg_lambda)

    def _calc_gain(self, G_left, H_left, G_right, H_right, G_total, H_total):
        gain = 0.5 * (G_left*G_left/(H_left + self.reg_lambda) + 
                     G_right*G_right/(H_right + self.reg_lambda) - 
                     G_total*G_total/(H_total + self.reg_lambda)) - self.gamma
        return gain

    def _find_best_split(self, X, g, h, sample_indices, feature_thresholds):
        G_total = np.sum(g[sample_indices])
        H_total = np.sum(h[sample_indices])
        
        best_gain = -np.inf
        best_feature = None
        best_threshold = None

        # Only check random subset of features (like Random Forest)
        n_features = len(feature_thresholds)
        n_features_to_check = int(np.sqrt(n_features))  # Random Forest style
        
        feature_indices = np.random.choice(n_features, n_features_to_check, replace=False)
        
        for feat_idx in feature_indices:
            thresholds = feature_thresholds[feat_idx]
            if thresholds is None or len(thresholds) == 0:
                continue

            x_col = X[sample_indices, feat_idx]
            g_vals = g[sample_indices]
            h_vals = h[sample_indices]

            # Sort only once per feature
            sorted_idx = np.argsort(x_col)
            x_sorted = x_col[sorted_idx]
            g_sorted = g_vals[sorted_idx]
            h_sorted = h_vals[sorted_idx]

            G_left, H_left = 0.0, 0.0
            
            # Try fewer thresholds for speed
            step = max(1, len(thresholds) // 10)  # Check only 10 thresholds per feature
            for th in thresholds[::step]:
                # Find split position efficiently
                split_pos = np.searchsorted(x_sorted, th, side='right')
                if split_pos == 0 or split_pos == len(x_sorted):
                    continue

                # Update cumulative sums incrementally
                if split_pos > len(g_sorted) // 2:
                    # Calculate from the right side for better numerical stability
                    G_right = np.sum(g_sorted[split_pos:])
                    H_right = np.sum(h_sorted[split_pos:])
                    G_left = G_total - G_right
                    H_left = H_total - H_right
                else:
                    G_left = np.sum(g_sorted[:split_pos])
                    H_left = np.sum(h_sorted[:split_pos])
                    G_right = G_total - G_left
                    H_right = H_total - H_left

                gain = self._calc_gain(G_left, H_left, G_right, H_right, G_total, H_total)
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feat_idx
                    best_threshold = th

        return best_feature, best_threshold, best_gain

    def _fit_recursive(self, X, g, h, sample_indices, depth, feature_thresholds):
        n_samples = len(sample_indices)
        if n_samples == 0:
            return

        G = np.sum(g[sample_indices])
        H = np.sum(h[sample_indices])
        self.leaf_value = self._calc_leaf_value(G, H)

        # Stop early if no improvement
        if depth >= self.max_depth or n_samples < 20:
            return

        feature, threshold, gain = self._find_best_split(X, g, h, sample_indices, feature_thresholds)

        if feature is None or gain <= 0:
            return

        self.feature = feature
        self.threshold = threshold

        left_mask = X[sample_indices, feature] <= threshold
        left_indices = sample_indices[left_mask]
        right_indices = sample_indices[~left_mask]

        if len(left_indices) == 0 or len(right_indices) == 0:
            return

        # Grow left and right branches
        self.left = FastXGBoostTree(
            max_depth=self.max_depth,
            reg_lambda=self.reg_lambda,
            gamma=self.gamma,
            max_bins=self.max_bins
        )
        self.left._fit_recursive(X, g, h, left_indices, depth + 1, feature_thresholds)

        self.right = FastXGBoostTree(
            max_depth=self.max_depth,
            reg_lambda=self.reg_lambda,
            gamma=self.gamma,
            max_bins=self.max_bins
        )
        self.right._fit_recursive(X, g, h, right_indices, depth + 1, feature_thresholds)

    def fit(self, X, g, h, feature_thresholds):
        sample_indices = np.arange(X.shape[0])
        self._fit_recursive(X, g, h, sample_indices, 0, feature_thresholds)
        return self

    def predict(self, X):
        if self.feature is None:
            return np.full(X.shape[0], self.leaf_value)
        
        left_mask = X[:, self.feature] <= self.threshold
        preds = np.full(X.shape[0], self.leaf_value)
        
        if self.left:
            preds[left_mask] = self.left.predict(X[left_mask])
        if self.right:
            preds[~left_mask] = self.right.predict(X[~left_mask])
            
        return preds


class FastXGBoostClassifier:
    def __init__(self, n_estimators=10, learning_rate=0.1, max_depth=3, reg_lambda=1.0, gamma=0.0, max_bins=50, subsample=0.8, colsample=0.8):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.reg_lambda = reg_lambda
        self.gamma = gamma
        self.max_bins = max_bins
        self.subsample = subsample
        self.colsample = colsample  # Column subsampling
        self.trees = []
        self.base_score = 0.0

    def _sigmoid(self, x):
        # Faster sigmoid approximation
        x = np.clip(x, -20, 20)  # Tighter bounds for stability
        return 0.5 * (x / (1 + np.abs(x))) + 0.5

    def _precompute_feature_thresholds(self, X):
        thresholds = []
        n_samples = X.shape[0]
        
        for feat in range(X.shape[1]):
            col = X[:, feat]
            if np.all(col == col[0]):
                thresholds.append(None)
                continue
                
            # Use fewer bins for speed
            n_bins = min(self.max_bins, n_samples // 10)
            if len(np.unique(col)) > n_bins:
                percentiles = np.linspace(10, 90, n_bins - 1)  # Skip extremes
                ths = np.percentile(col, percentiles)
            else:
                unique_vals = np.unique(col)
                ths = (unique_vals[:-1] + unique_vals[1:]) / 2.0
                
            thresholds.append(ths if len(ths) > 0 else None)
        return thresholds

    def fit(self, X, y):
        X = np.asarray(X, dtype=np.float32)  # Use float32 for speed
        y = np.asarray(y, dtype=np.float32)
        y_binary = y

        # Precompute global thresholds once
        feature_thresholds = self._precompute_feature_thresholds(X)

        n_samples, n_features = X.shape
        y_pred = np.full(n_samples, self.base_score, dtype=np.float32)

        for i in range(self.n_estimators):
            # Row subsampling
            if self.subsample < 1.0:
                subsample_size = int(self.subsample * n_samples)
                row_indices = np.random.choice(n_samples, subsample_size, replace=False)
            else:
                row_indices = np.arange(n_samples)

            # Column subsampling for extra speed and regularization
            if self.colsample < 1.0:
                n_cols = int(self.colsample * n_features)
                col_indices = np.random.choice(n_features, n_cols, replace=False)
                X_sub = X[row_indices][:, col_indices]
                feature_thresholds_sub = [feature_thresholds[i] for i in col_indices]
            else:
                col_indices = np.arange(n_features)
                X_sub = X[row_indices]
                feature_thresholds_sub = feature_thresholds

            y_sub = y_binary[row_indices]
            y_pred_sub = y_pred[row_indices]

            # Calculate gradients and hessians
            p = self._sigmoid(y_pred_sub)
            g = p - y_sub
            h = p * (1.0 - p) + 1e-6  # Smaller epsilon

            # Train tree
            tree = FastXGBoostTree(
                max_depth=self.max_depth,
                reg_lambda=self.reg_lambda,
                gamma=self.gamma,
                max_bins=self.max_bins
            )
            tree.fit(X_sub, g, h, feature_thresholds_sub)
            
            # Store column indices for prediction
            tree.col_indices = col_indices
            self.trees.append(tree)

            # Update predictions
            if self.colsample < 1.0:
                X_pred = X[:, col_indices]
            else:
                X_pred = X
                
            update = tree.predict(X_pred)
            y_pred += self.learning_rate * update

        return self

    def predict(self, X):
        X = np.asarray(X, dtype=np.float32)
        y_pred = np.full(X.shape[0], self.base_score, dtype=np.float32)
        
        for tree in self.trees:
            if hasattr(tree, 'col_indices'):
                X_sub = X[:, tree.col_indices]
            else:
                X_sub = X
            y_pred += self.learning_rate * tree.predict(X_sub)
            
        proba = self._sigmoid(y_pred)
        return (proba >= 0.5).astype(int)


class FastMultiClassXGBoost:
    def __init__(self, n_estimators=10, learning_rate=0.1, max_depth=3, reg_lambda=1.0, 
                 gamma=0.0, max_bins=50, n_classes=10, subsample=0.8, colsample=0.8):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.reg_lambda = reg_lambda
        self.gamma = gamma
        self.max_bins = max_bins
        self.n_classes = n_classes
        self.subsample = subsample
        self.colsample = colsample
        self.classifiers = []

    def fit(self, X, y):
        self.classifiers = []
        
        for class_idx in range(self.n_classes):
            print(f"Training XGBoost for class {class_idx+1}/{self.n_classes}...")
            
            y_binary = (y == class_idx).astype(int)
            
            xgb_binary = FastXGBoostClassifier(
                n_estimators=self.n_estimators,
                learning_rate=self.learning_rate,
                max_depth=self.max_depth,
                reg_lambda=self.reg_lambda,
                gamma=self.gamma,
                max_bins=self.max_bins,
                subsample=self.subsample,
                colsample=self.colsample
            )
            
            xgb_binary.fit(X, y_binary)
            self.classifiers.append(xgb_binary)
            
        return self

    def predict_proba(self, X):
        probabilities = np.zeros((X.shape[0], self.n_classes), dtype=np.float32)
        
        for class_idx, classifier in enumerate(self.classifiers):
            raw_scores = np.zeros(X.shape[0], dtype=np.float32)
            for tree in classifier.trees:
                if hasattr(tree, 'col_indices'):
                    X_sub = X[:, tree.col_indices]
                else:
                    X_sub = X
                raw_scores += classifier.learning_rate * tree.predict(X_sub)
            
            # Use fast sigmoid
            proba = 0.5 * (raw_scores / (1 + np.abs(raw_scores))) + 0.5
            probabilities[:, class_idx] = proba
            
        # Normalize
        prob_sums = np.sum(probabilities, axis=1, keepdims=True)
        probabilities /= np.maximum(prob_sums, 1e-8)
        return probabilities

    def predict(self, X):
        probabilities = self.predict_proba(X)
        return np.argmax(probabilities, axis=1)


# Test OPTIMIZED XGBoost
print("="*60)
print("OPTIMIZED XGBOOST TESTING")
print("="*60)

# Optimized settings for speed AND accuracy
xgb_fast = FastMultiClassXGBoost(
    n_estimators=60,           # Balanced number
    learning_rate=0.15,        # Slightly higher for faster convergence
    max_depth=7,               # Optimal depth
    reg_lambda=0.8,            # Slight regularization
    subsample=0.7,             # Good balance
    colsample=0.4,             # Column subsampling for extra speed
    max_bins=32,               # Faster with good accuracy
    n_classes=10
)

# Training
print("\n🚀 Starting OPTIMIZED XGBoost training...")
train_start = time.time()
xgb_fast.fit(X_train, y_train)
training_time = time.time() - train_start
print(f"✅ Training completed in {training_time:.2f} seconds")

# Prediction
print("\n🎯 Making predictions...")
pred_start = time.time()
y_pred_xgb = xgb_fast.predict(X_val)
pred_time = time.time() - pred_start

print(f"✅ Prediction completed in {pred_time:.4f} seconds")

# Calculate metrics
f1_weighted_xgb = f1_score(y_val, y_pred_xgb, average='weighted')
accuracy_xgb = np.mean(y_pred_xgb == y_val)

print(f"\n📊 OPTIMIZED XGBoost Performance:")
print(f"F1 Score (Weighted): {f1_weighted_xgb:.4f}")
print(f"Accuracy: {accuracy_xgb:.4f}")
print(f"Total Time: {training_time + pred_time:.2f}s")
print(f"Expected F1: 0.92-0.95")

OPTIMIZED XGBOOST TESTING

🚀 Starting OPTIMIZED XGBoost training...
Training XGBoost for class 1/10...
Training XGBoost for class 2/10...
Training XGBoost for class 3/10...
Training XGBoost for class 4/10...
Training XGBoost for class 5/10...
Training XGBoost for class 6/10...
Training XGBoost for class 7/10...
Training XGBoost for class 8/10...
Training XGBoost for class 9/10...
Training XGBoost for class 10/10...
✅ Training completed in 140.06 seconds

🎯 Making predictions...
✅ Prediction completed in 7.5886 seconds

📊 OPTIMIZED XGBoost Performance:
F1 Score (Weighted): 0.9351
Accuracy: 0.9352
Total Time: 147.65s
Expected F1: 0.92-0.95


0.8
30,3,78.86,0.85
30,4,90.82,0.87
20,4,85.56,0.86
30,5,100.73,0.9082
40,5,203.39,0.9163
50,5,154.95,0.9198
50,6,250.25,0.9294
0.6
50,6,108.32,0.9246
0.4
60,6,107.46,0.9316

In [108]:
class MultiClassSVM:
    def __init__(self, learning_rate=0.001, lambda_param=0.01, n_iters=1000, n_classes=10):
        self.lr = learning_rate
        self.lambda_param = lambda_param
        self.n_iters = n_iters
        self.n_classes = n_classes
        self.W = None  # Weight matrix for all classes
        self.b = None  # Bias vector for all classes

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.W = np.zeros((self.n_classes, n_features))
        self.b = np.zeros(self.n_classes)

        # One-vs-Rest approach
        for class_idx in range(self.n_classes):
            print(f"Training SVM for class {class_idx}...")
            # Convert to binary labels: current class vs all others
            y_binary = np.where(y == class_idx, 1, -1)
            
            w = np.zeros(n_features)
            b = 0

            for _ in range(self.n_iters):
                for i, x_i in enumerate(X):
                    condition = y_binary[i] * (np.dot(x_i, w) - b) >= 1
                    if condition:
                        w -= self.lr * (self.lambda_param * w)
                    else:
                        w -= self.lr * (self.lambda_param * w - np.dot(x_i, y_binary[i]))
                        b -= self.lr * y_binary[i]

            self.W[class_idx] = w
            self.b[class_idx] = b

    def predict(self, X):
        # For each sample, find the class with highest score
        scores = np.dot(X, self.W.T) - self.b
        return np.argmax(scores, axis=1)

# Test SVM
print("="*60)
print("SVM MODEL TESTING - ONE vs REST")
print("="*60)

# Initialize SVM with fewer iterations for speed
svm = MultiClassSVM(
    learning_rate=0.001,
    lambda_param=0.01, 
    n_iters=100,  # Reduced for speed
    n_classes=10
)

# Training
print("\n🚀 Starting SVM training (this may take a while)...")
train_start = time.time()
svm.fit(X_train, y_train)
training_time = time.time() - train_start
print(f"✅ Training completed in {training_time:.2f} seconds")

# Prediction
print("\n🎯 Making predictions on validation set...")
pred_start = time.time()
y_pred_svm = svm.predict(X_val)
pred_time = time.time() - pred_start

print(f"✅ Prediction completed in {pred_time:.4f} seconds")

SVM MODEL TESTING - ONE vs REST

🚀 Starting SVM training (this may take a while)...
Training SVM for class 0...
Training SVM for class 1...
Training SVM for class 2...
Training SVM for class 3...
Training SVM for class 4...
Training SVM for class 5...
Training SVM for class 6...
Training SVM for class 7...
Training SVM for class 8...
Training SVM for class 9...
✅ Training completed in 49.50 seconds

🎯 Making predictions on validation set...
✅ Prediction completed in 0.0161 seconds


In [109]:
# Calculate metrics
print("\n" + "="*60)
print("SVM PERFORMANCE METRICS")
print("="*60)

f1_macro_svm = f1_score(y_val, y_pred_svm, average='macro')
f1_weighted_svm = f1_score(y_val, y_pred_svm, average='weighted')
accuracy_svm = np.mean(y_pred_svm == y_val)

print(f"F1 Score (Macro): {f1_macro_svm:.4f}")
print(f"F1 Score (Weighted): {f1_weighted_svm:.4f}")
print(f"Accuracy: {accuracy_svm:.4f}")
print(f"Training Time: {training_time:.2f}s")
print(f"Prediction Time: {pred_time:.4f}s")

# Compare with all models
print(f"\n🆚 COMPARISON WITH ALL MODELS:")
print(f"KNN F1:          0.9523")
print(f"Softmax F1:      0.8899")
print(f"Random Forest F1: 0.8503")
print(f"Naive Bayes F1:   0.5424")
print(f"SVM F1:          {f1_weighted_svm:.4f}")

# Time analysis
total_svm_time = training_time + pred_time
total_used = 8.11 + 163.38 + 134.94 + total_svm_time
remaining_time = 300 - total_used
print(f"\n⏰ REMAINING TIME BUDGET: {remaining_time:.2f}s ({remaining_time/60:.2f}min)")


SVM PERFORMANCE METRICS
F1 Score (Macro): 0.8922
F1 Score (Weighted): 0.8934
Accuracy: 0.8948
Training Time: 49.50s
Prediction Time: 0.0161s

🆚 COMPARISON WITH ALL MODELS:
KNN F1:          0.9523
Softmax F1:      0.8899
Random Forest F1: 0.8503
Naive Bayes F1:   0.5424
SVM F1:          0.8934

⏰ REMAINING TIME BUDGET: -55.95s (-0.93min)


In [112]:
# Hyperparameter combinations to try (ordered by likely performance)
param_combinations = [
    # Fastest first, then more accurate but slower
    {'learning_rate': 0.01, 'lambda_param': 0.1, 'n_iters': 30},
    {'learning_rate': 0.01, 'lambda_param': 0.01, 'n_iters': 40},
    {'learning_rate': 0.005, 'lambda_param': 0.05, 'n_iters': 50},
    {'learning_rate': 0.005, 'lambda_param': 0.01, 'n_iters': 60},
    {'learning_rate': 0.001, 'lambda_param': 0.01, 'n_iters': 80},
]
svm = MultiClassSVM(
    learning_rate=0.01,
    lambda_param=0.01, 
    n_iters=30,  # Reduced for speed
    n_classes=10
)

# Training
print("\n🚀 Starting SVM training (this may take a while)...")
train_start = time.time()
svm.fit(X_train, y_train)
training_time = time.time() - train_start
print(f"✅ Training completed in {training_time:.2f} seconds")

# Prediction
print("\n🎯 Making predictions on validation set...")
pred_start = time.time()
y_pred_svm = svm.predict(X_val)
pred_time = time.time() - pred_start

print(f"✅ Prediction completed in {pred_time:.4f} seconds")
f1_weighted_svm = f1_score(y_val, y_pred_svm, average='weighted')

print(f"F1 Score (Weighted): {f1_weighted_svm:.4f}")


🚀 Starting SVM training (this may take a while)...
Training SVM for class 0...
Training SVM for class 1...
Training SVM for class 2...
Training SVM for class 3...
Training SVM for class 4...
Training SVM for class 5...
Training SVM for class 6...
Training SVM for class 7...
Training SVM for class 8...
Training SVM for class 9...
✅ Training completed in 15.45 seconds

🎯 Making predictions on validation set...
✅ Prediction completed in 0.0022 seconds
F1 Score (Weighted): 0.8575


In [113]:
# Hyperparameter combinations to try (ordered by likely performance)
param_combinations = [
    # Fastest first, then more accurate but slower
    {'learning_rate': 0.01, 'lambda_param': 0.1, 'n_iters': 30},
    {'learning_rate': 0.01, 'lambda_param': 0.01, 'n_iters': 40},
    {'learning_rate': 0.005, 'lambda_param': 0.05, 'n_iters': 50},
    {'learning_rate': 0.005, 'lambda_param': 0.01, 'n_iters': 60},
    {'learning_rate': 0.001, 'lambda_param': 0.01, 'n_iters': 80},
]
svm = MultiClassSVM(
    learning_rate=0.01,
    lambda_param=0.01, 
    n_iters=40,  # Reduced for speed
    n_classes=10
)

# Training
print("\n🚀 Starting SVM training (this may take a while)...")
train_start = time.time()
svm.fit(X_train, y_train)
training_time = time.time() - train_start
print(f"✅ Training completed in {training_time:.2f} seconds")

# Prediction
print("\n🎯 Making predictions on validation set...")
pred_start = time.time()
y_pred_svm = svm.predict(X_val)
pred_time = time.time() - pred_start

print(f"✅ Prediction completed in {pred_time:.4f} seconds")
f1_weighted_svm = f1_score(y_val, y_pred_svm, average='weighted')

print(f"F1 Score (Weighted): {f1_weighted_svm:.4f}")


🚀 Starting SVM training (this may take a while)...
Training SVM for class 0...
Training SVM for class 1...
Training SVM for class 2...
Training SVM for class 3...
Training SVM for class 4...
Training SVM for class 5...
Training SVM for class 6...
Training SVM for class 7...
Training SVM for class 8...
Training SVM for class 9...
✅ Training completed in 21.45 seconds

🎯 Making predictions on validation set...
✅ Prediction completed in 0.0035 seconds
F1 Score (Weighted): 0.8605


In [114]:
# Hyperparameter combinations to try (ordered by likely performance)
param_combinations = [
    # Fastest first, then more accurate but slower
    {'learning_rate': 0.01, 'lambda_param': 0.1, 'n_iters': 30},
    {'learning_rate': 0.01, 'lambda_param': 0.01, 'n_iters': 40},
    {'learning_rate': 0.005, 'lambda_param': 0.05, 'n_iters': 50},
    {'learning_rate': 0.005, 'lambda_param': 0.01, 'n_iters': 60},
    {'learning_rate': 0.001, 'lambda_param': 0.01, 'n_iters': 80},
]
svm = MultiClassSVM(
    learning_rate=0.005,
    lambda_param=0.05, 
    n_iters=50,  # Reduced for speed
    n_classes=10
)

# Training
print("\n🚀 Starting SVM training (this may take a while)...")
train_start = time.time()
svm.fit(X_train, y_train)
training_time = time.time() - train_start
print(f"✅ Training completed in {training_time:.2f} seconds")

# Prediction
print("\n🎯 Making predictions on validation set...")
pred_start = time.time()
y_pred_svm = svm.predict(X_val)
pred_time = time.time() - pred_start

print(f"✅ Prediction completed in {pred_time:.4f} seconds")
f1_weighted_svm = f1_score(y_val, y_pred_svm, average='weighted')

print(f"F1 Score (Weighted): {f1_weighted_svm:.4f}")


🚀 Starting SVM training (this may take a while)...
Training SVM for class 0...
Training SVM for class 1...
Training SVM for class 2...
Training SVM for class 3...
Training SVM for class 4...
Training SVM for class 5...
Training SVM for class 6...
Training SVM for class 7...
Training SVM for class 8...
Training SVM for class 9...
✅ Training completed in 26.44 seconds

🎯 Making predictions on validation set...
✅ Prediction completed in 0.0020 seconds
F1 Score (Weighted): 0.8271


In [116]:
# Hyperparameter combinations to try (ordered by likely performance)
param_combinations = [
    # Fastest first, then more accurate but slower
    {'learning_rate': 0.01, 'lambda_param': 0.1, 'n_iters': 30},
    {'learning_rate': 0.01, 'lambda_param': 0.01, 'n_iters': 40},
    {'learning_rate': 0.005, 'lambda_param': 0.05, 'n_iters': 50},
    {'learning_rate': 0.005, 'lambda_param': 0.01, 'n_iters': 60},
    {'learning_rate': 0.001, 'lambda_param': 0.01, 'n_iters': 80},
]
svm = MultiClassSVM(
    learning_rate=0.005,
    lambda_param=0.01, 
    n_iters=60,  # Reduced for speed
    n_classes=10
)

# Training
print("\n🚀 Starting SVM training (this may take a while)...")
train_start = time.time()
svm.fit(X_train, y_train)
training_time = time.time() - train_start
print(f"✅ Training completed in {training_time:.2f} seconds")

# Prediction
print("\n🎯 Making predictions on validation set...")
pred_start = time.time()
y_pred_svm = svm.predict(X_val)
pred_time = time.time() - pred_start

print(f"✅ Prediction completed in {pred_time:.4f} seconds")
f1_weighted_svm = f1_score(y_val, y_pred_svm, average='weighted')

print(f"F1 Score (Weighted): {f1_weighted_svm:.4f}")


🚀 Starting SVM training (this may take a while)...
Training SVM for class 0...
Training SVM for class 1...
Training SVM for class 2...
Training SVM for class 3...
Training SVM for class 4...
Training SVM for class 5...
Training SVM for class 6...
Training SVM for class 7...
Training SVM for class 8...
Training SVM for class 9...
✅ Training completed in 38.12 seconds

🎯 Making predictions on validation set...
✅ Prediction completed in 0.0000 seconds
F1 Score (Weighted): 0.8752


In [117]:
# Hyperparameter combinations to try (ordered by likely performance)
param_combinations = [
    # Fastest first, then more accurate but slower
    {'learning_rate': 0.01, 'lambda_param': 0.1, 'n_iters': 30},
    {'learning_rate': 0.01, 'lambda_param': 0.01, 'n_iters': 40},
    {'learning_rate': 0.005, 'lambda_param': 0.05, 'n_iters': 50},
    {'learning_rate': 0.005, 'lambda_param': 0.01, 'n_iters': 60},
    {'learning_rate': 0.001, 'lambda_param': 0.01, 'n_iters': 80},
]
svm = MultiClassSVM(
    learning_rate=0.001,
    lambda_param=0.01, 
    n_iters=80,  # Reduced for speed
    n_classes=10
)

# Training
print("\n🚀 Starting SVM training (this may take a while)...")
train_start = time.time()
svm.fit(X_train, y_train)
training_time = time.time() - train_start
print(f"✅ Training completed in {training_time:.2f} seconds")

# Prediction
print("\n🎯 Making predictions on validation set...")
pred_start = time.time()
y_pred_svm = svm.predict(X_val)
pred_time = time.time() - pred_start

print(f"✅ Prediction completed in {pred_time:.4f} seconds")
f1_weighted_svm = f1_score(y_val, y_pred_svm, average='weighted')

print(f"F1 Score (Weighted): {f1_weighted_svm:.4f}")


🚀 Starting SVM training (this may take a while)...
Training SVM for class 0...
Training SVM for class 1...
Training SVM for class 2...
Training SVM for class 3...
Training SVM for class 4...
Training SVM for class 5...
Training SVM for class 6...
Training SVM for class 7...
Training SVM for class 8...
Training SVM for class 9...
✅ Training completed in 50.37 seconds

🎯 Making predictions on validation set...
✅ Prediction completed in 0.0000 seconds
F1 Score (Weighted): 0.8930


In [118]:
# Hyperparameter combinations to try (ordered by likely performance)
param_combinations = [
    # Fastest first, then more accurate but slower
    {'learning_rate': 0.01, 'lambda_param': 0.1, 'n_iters': 30},
    {'learning_rate': 0.01, 'lambda_param': 0.01, 'n_iters': 40},
    {'learning_rate': 0.005, 'lambda_param': 0.05, 'n_iters': 50},
    {'learning_rate': 0.005, 'lambda_param': 0.01, 'n_iters': 60},
    {'learning_rate': 0.001, 'lambda_param': 0.01, 'n_iters': 80},
]
svm = MultiClassSVM(
    learning_rate=0.0005,
    lambda_param=0.01, 
    n_iters=100,  # Reduced for speed
    n_classes=10
)

# Training
print("\n🚀 Starting SVM training (this may take a while)...")
train_start = time.time()
svm.fit(X_train, y_train)
training_time = time.time() - train_start
print(f"✅ Training completed in {training_time:.2f} seconds")

# Prediction
print("\n🎯 Making predictions on validation set...")
pred_start = time.time()
y_pred_svm = svm.predict(X_val)
pred_time = time.time() - pred_start

print(f"✅ Prediction completed in {pred_time:.4f} seconds")
f1_weighted_svm = f1_score(y_val, y_pred_svm, average='weighted')

print(f"F1 Score (Weighted): {f1_weighted_svm:.4f}")


🚀 Starting SVM training (this may take a while)...
Training SVM for class 0...
Training SVM for class 1...
Training SVM for class 2...
Training SVM for class 3...
Training SVM for class 4...
Training SVM for class 5...
Training SVM for class 6...
Training SVM for class 7...
Training SVM for class 8...
Training SVM for class 9...
✅ Training completed in 57.38 seconds

🎯 Making predictions on validation set...
✅ Prediction completed in 0.0000 seconds
F1 Score (Weighted): 0.8957


In [119]:
# Hyperparameter combinations to try (ordered by likely performance)
param_combinations = [
    # Fastest first, then more accurate but slower
    {'learning_rate': 0.01, 'lambda_param': 0.1, 'n_iters': 30},
    {'learning_rate': 0.01, 'lambda_param': 0.01, 'n_iters': 40},
    {'learning_rate': 0.005, 'lambda_param': 0.05, 'n_iters': 50},
    {'learning_rate': 0.005, 'lambda_param': 0.01, 'n_iters': 60},
    {'learning_rate': 0.001, 'lambda_param': 0.01, 'n_iters': 80},
]
svm = MultiClassSVM(
    learning_rate=0.0003,
    lambda_param=0.008, 
    n_iters=120,  # Reduced for speed
    n_classes=10
)

# Training
print("\n🚀 Starting SVM training (this may take a while)...")
train_start = time.time()
svm.fit(X_train, y_train)
training_time = time.time() - train_start
print(f"✅ Training completed in {training_time:.2f} seconds")

# Prediction
print("\n🎯 Making predictions on validation set...")
pred_start = time.time()
y_pred_svm = svm.predict(X_val)
pred_time = time.time() - pred_start

print(f"✅ Prediction completed in {pred_time:.4f} seconds")
f1_weighted_svm = f1_score(y_val, y_pred_svm, average='weighted')

print(f"F1 Score (Weighted): {f1_weighted_svm:.4f}")


🚀 Starting SVM training (this may take a while)...
Training SVM for class 0...
Training SVM for class 1...
Training SVM for class 2...
Training SVM for class 3...
Training SVM for class 4...
Training SVM for class 5...
Training SVM for class 6...
Training SVM for class 7...
Training SVM for class 8...
Training SVM for class 9...
✅ Training completed in 65.63 seconds

🎯 Making predictions on validation set...
✅ Prediction completed in 0.0068 seconds
F1 Score (Weighted): 0.9023


In [120]:
import numpy as np
import time
from collections import Counter
from sklearn.metrics import f1_score, classification_report

class PCA:
    def __init__(self, n_components):
        self.n_components = n_components
        self.components = None
        self.mean = None
        
    def fit(self, X):
        # Center the data
        self.mean = np.mean(X, axis=0)
        X_centered = X - self.mean
        
        # Compute covariance matrix
        cov_matrix = np.cov(X_centered, rowvar=False)
        
        # Compute eigenvalues and eigenvectors
        eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)
        
        # Sort eigenvectors by eigenvalues in descending order
        sorted_indices = np.argsort(eigenvalues)[::-1]
        self.components = eigenvectors[:, sorted_indices[:self.n_components]]
        
    def transform(self, X):
        X_centered = X - self.mean
        return np.dot(X_centered, self.components)

class PCAWeightedKNN:
    def __init__(self, k=5, n_components=50, batch_size=100):
        self.k = k
        self.n_components = n_components
        self.batch_size = batch_size
        self.pca = PCA(n_components)
        self.X_train_pca = None
        self.y_train = None
        
    def fit(self, X, y):
        print(f"Applying PCA to reduce dimensions from {X.shape[1]} to {self.n_components}...")
        start_pca = time.time()
        self.pca.fit(X)
        self.X_train_pca = self.pca.transform(X.astype(np.float32))
        self.y_train = y.astype(np.int32)
        pca_time = time.time() - start_pca
        print(f"PCA completed in {pca_time:.2f}s")
        
    def predict(self, X):
        X_pca = self.pca.transform(X.astype(np.float32))
        predictions = np.empty(X_pca.shape[0], dtype=np.int32)
        
        # Precompute squared norms for training data in PCA space
        train_norms = np.sum(self.X_train_pca ** 2, axis=1)
        
        # Process in batches
        for batch_start in range(0, X_pca.shape[0], self.batch_size):
            batch_end = min(batch_start + self.batch_size, X_pca.shape[0])
            X_batch = X_pca[batch_start:batch_end]
            
            # Vectorized distance computation in PCA space
            test_norms = np.sum(X_batch ** 2, axis=1)
            dot_products = np.dot(self.X_train_pca, X_batch.T)
            squared_distances = train_norms[:, np.newaxis] + test_norms - 2 * dot_products
            
            # Get k nearest neighbors for each batch sample
            nearest_indices = np.argpartition(squared_distances, self.k, axis=0)[:self.k]
            nearest_squared_distances = np.take_along_axis(squared_distances, nearest_indices, axis=0)
            nearest_labels = self.y_train[nearest_indices]
            
            # Convert to actual distances and compute weights
            distances = np.sqrt(nearest_squared_distances + 1e-8)
            weights = 1.0 / (distances + 1e-8)
            
            # Weighted majority voting for each sample in batch
            for j in range(batch_end - batch_start):
                label_weights = {}
                for idx in range(self.k):
                    label = nearest_labels[idx, j]
                    weight = weights[idx, j]
                    label_weights[label] = label_weights.get(label, 0.0) + weight
                
                predictions[batch_start + j] = max(label_weights.items(), key=lambda x: x[1])[0]
                
        return predictions

# Test different PCA dimensions to find optimal
def find_optimal_pca_knn(X_train, y_train, X_val, y_val):
    print("="*60)
    print("FINDING OPTIMAL PCA DIMENSIONS FOR WEIGHTED KNN")
    print("="*60)
    
    pca_dims = [30, 50, 80, 100, 150]  # Test different dimensions
    best_f1 = 0
    best_pca_dim = 0
    best_knn = None
    
    for n_components in pca_dims:
        print(f"\nTesting PCA with {n_components} components...")
        
        knn_pca = PCAWeightedKNN(k=5, n_components=n_components, batch_size=200)
        
        # Training (includes PCA)
        start_time = time.time()
        knn_pca.fit(X_train, y_train)
        training_time = time.time() - start_time
        
        # Prediction
        pred_start = time.time()
        y_pred = knn_pca.predict(X_val)
        pred_time = time.time() - pred_start
        
        f1 = f1_score(y_val, y_pred, average='weighted')
        total_time = training_time + pred_time
        
        print(f"  F1 Score: {f1:.4f}")
        print(f"  Total Time: {total_time:.2f}s")
        print(f"  Dimensions: {X_train.shape[1]} → {n_components}")
        
        if f1 > best_f1:
            best_f1 = f1
            best_pca_dim = n_components
            best_knn = knn_pca
            print(f"  🎯 NEW BEST!")
    
    print(f"\n" + "="*50)
    print(f"OPTIMAL PCA DIMENSIONS: {best_pca_dim}")
    print(f"BEST F1 SCORE: {best_f1:.4f}")
    print("="*50)
    
    return best_knn, best_pca_dim, best_f1

# Test with optimal PCA
print("Testing PCA-WeightedKNN with optimal dimensions...")
optimal_knn, optimal_dim, optimal_f1 = find_optimal_pca_knn(X_train, y_train, X_val, y_val)

# Final performance with best PCA
print("\n" + "="*60)
print("FINAL PCA-WEIGHTED KNN PERFORMANCE")
print("="*60)

# One more run with optimal parameters for accurate timing
final_knn = PCAWeightedKNN(k=5, n_components=optimal_dim, batch_size=200)

start_time = time.time()
final_knn.fit(X_train, y_train)
training_time = time.time() - start_time

pred_start = time.time()
y_pred_final = final_knn.predict(X_val)
pred_time = time.time() - pred_start

f1_final = f1_score(y_val, y_pred_final, average='weighted')
accuracy_final = np.mean(y_pred_final == y_val)

print(f"F1 Score: {f1_final:.4f}")
print(f"Accuracy: {accuracy_final:.4f}")
print(f"Training Time (incl PCA): {training_time:.2f}s")
print(f"Prediction Time: {pred_time:.2f}s")
print(f"Total Time: {training_time + pred_time:.2f}s")
print(f"PCA Dimensions: {X_train.shape[1]} → {optimal_dim}")
print(f"Speedup vs original: ~{163.38/(training_time + pred_time):.1f}x faster!")

# Compare with original KNN
print(f"\nComparison with original KNN:")
print(f"Original KNN F1: 0.9523")
print(f"PCA KNN F1:      {f1_final:.4f}")
print(f"Difference:      {f1_final - 0.9523:+.4f}")

Testing PCA-WeightedKNN with optimal dimensions...
FINDING OPTIMAL PCA DIMENSIONS FOR WEIGHTED KNN

Testing PCA with 30 components...
Applying PCA to reduce dimensions from 784 to 30...
PCA completed in 0.32s
  F1 Score: 0.9583
  Total Time: 0.84s
  Dimensions: 784 → 30
  🎯 NEW BEST!

Testing PCA with 50 components...
Applying PCA to reduce dimensions from 784 to 50...
PCA completed in 0.32s
  F1 Score: 0.9579
  Total Time: 0.95s
  Dimensions: 784 → 50

Testing PCA with 80 components...
Applying PCA to reduce dimensions from 784 to 80...
PCA completed in 0.31s
  F1 Score: 0.9571
  Total Time: 0.84s
  Dimensions: 784 → 80

Testing PCA with 100 components...
Applying PCA to reduce dimensions from 784 to 100...
PCA completed in 0.29s
  F1 Score: 0.9539
  Total Time: 0.82s
  Dimensions: 784 → 100

Testing PCA with 150 components...
Applying PCA to reduce dimensions from 784 to 150...
PCA completed in 0.29s
  F1 Score: 0.9535
  Total Time: 0.89s
  Dimensions: 784 → 150

OPTIMAL PCA DIMENSIO